# Car Price Prediction: Ridge & Lasso

En este ejercicio tenemos como objetivo predecir el precio de un auto en el mercado estadounidense, entendiendo qué variables son más o menos determinantes a la hora de hacerlo.

Primero, cargamos los datos y visualizamos las primeras cinco filas:

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/car-price-prediction/CarPrice_Assignment.csv")

print("First 5 records:", df.head())

Primeramente identificamos unas cuantas variables categóricas que tendremos que one-hot-encodear. 

Seguimos analizando los datos:

In [ ]:
df.describe()

In [ ]:
df.info()

Lo primero que identificamos es que no hay valores nulos, lo cual, en principio, es bueno.

Vamos a graficar una matriz de correlación solo usando las variables numéricas:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_numerical = df.drop(["car_ID", "CarName", "fueltype", "aspiration", "doornumber", "carbody", "drivewheel", "enginelocation", "enginetype", "cylindernumber", "fuelsystem"], axis=1)

corr = df_numerical.corr()
plt.figure(figsize=(6, 4))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title("Matriz de correlación")
plt.show()

Podemos ver que la feature más correlacionada con `price` es `enginesize`, seguida por `curbweight` y `horsepower`. 

Vamos a probar unos cuantos modelos para ver cual tomamos como baseline:

- Regresión lineal usando `enginesize` como *X*.
- Regresión múltiple usando `enginesize`, `curbweight` y `horsepower`.
- Regresión múltiple usando *todas* las variables luego de one-hot-encodear las categóricas.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Primer modelo

y = df["price"]

X_1 = df[["enginesize"]]
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y, test_size=0.25, random_state=42)

model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)

# Segundo modelo

X_2 = df[["enginesize", "curbweight", "horsepower"]]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y, test_size=0.25, random_state=42)

model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2)

# Tercer modelo

X_3 = df.drop("price", axis=1)

categorical_cols = X_3.select_dtypes(include=['object']).columns
numeric_cols = X_3.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
    ], remainder='passthrough')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y, test_size=0.25, random_state=42)

pipeline.fit(X_train_3, y_train_3)

Luego, evaluamos y comparamos estos tres modelos:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Diccionarios para almacenar métricas
metrics = {'Modelo': [], 'R2_train': [], 'R2_test': [], 'MSE_train': [], 'MSE_test': []}

# --- Modelo 1 ---
y_pred_train_1 = model_1.predict(X_train_1)
y_pred_test_1 = model_1.predict(X_test_1)

metrics['Modelo'].append('Model 1 (enginesize)')
metrics['R2_train'].append(r2_score(y_train_1, y_pred_train_1))
metrics['R2_test'].append(r2_score(y_test_1, y_pred_test_1))
metrics['MSE_train'].append(mean_squared_error(y_train_1, y_pred_train_1))
metrics['MSE_test'].append(mean_squared_error(y_test_1, y_pred_test_1))

# --- Modelo 2 ---
y_pred_train_2 = model_2.predict(X_train_2)
y_pred_test_2 = model_2.predict(X_test_2)

metrics['Modelo'].append('Model 2 (3 num)')
metrics['R2_train'].append(r2_score(y_train_2, y_pred_train_2))
metrics['R2_test'].append(r2_score(y_test_2, y_pred_test_2))
metrics['MSE_train'].append(mean_squared_error(y_train_2, y_pred_train_2))
metrics['MSE_test'].append(mean_squared_error(y_test_2, y_pred_test_2))

# --- Modelo 3 (pipeline con todas las variables) ---
y_pred_train_3 = pipeline.predict(X_train_3)
y_pred_test_3 = pipeline.predict(X_test_3)

metrics['Modelo'].append('Model 3 (todas)')
metrics['R2_train'].append(r2_score(y_train_3, y_pred_train_3))
metrics['R2_test'].append(r2_score(y_test_3, y_pred_test_3))
metrics['MSE_train'].append(mean_squared_error(y_train_3, y_pred_train_3))
metrics['MSE_test'].append(mean_squared_error(y_test_3, y_pred_test_3))

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

# --- Graficos comparativos ---
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# R2 plot
axes[0].bar(metrics_df['Modelo'], metrics_df['R2_train'], alpha=0.7, label='Train')
axes[0].bar(metrics_df['Modelo'], metrics_df['R2_test'], alpha=0.7, label='Test')
axes[0].set_ylim(0,1)
axes[0].set_title('R² Train vs Test')
axes[0].legend()
axes[0].set_ylabel('R²')

# MSE plot
axes[1].bar(metrics_df['Modelo'], metrics_df['MSE_train'], alpha=0.7, label='Train')
axes[1].bar(metrics_df['Modelo'], metrics_df['MSE_test'], alpha=0.7, label='Test')
axes[1].set_title('MSE Train vs Test')
axes[1].legend()
axes[1].set_ylabel('MSE')

plt.tight_layout()
plt.show()


- El modelo 1 es bueno, con un R2 bastante mejor en test que en train (puede ser casualidad debido a que el dataset no es muy grande) y MSE muy similares entre train y test.
- El segundo modelo es mejor al tener más información con la que trabajar, teniendo incluso menor MSE que el modelo 2.
- El último modelo, que considera **todas** las features, parece ser muy malo. Un R2 tan cercano a 1 en el train set y tan malo en el test set nos dice que hay un *overfitting enorme*. Esto tambien se refleja en el MSE, siendo abismalmente más grande que sus predecesores. Una muy probable causa de esto es la gran cantidad de variables categóricas one-hot-encodeadas, es decir, **el modelo es demasiado complejo para los datos disponibles**. Afortunadamente, podemos resolver este problema con **Ridge** y **Lasso**.

El modelo 2 es el mejor y el que tomaremos como baseline.

A continuación, entrenamos Ridge y Lasso:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso

X = df.drop("price", axis=1)
y = df["price"]

categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Escalado de variables numéricas
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X[numeric_cols])

# One-hot-encoding de categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
X_cat_encoded = encoder.fit_transform(X[categorical_cols])

import numpy as np
X_full = np.hstack([X_num_scaled, X_cat_encoded])

models = {
    "Ridge (todas)": Ridge(alpha=1.0),
    "Lasso (todas)": Lasso(alpha=1.0)
}

results = {'Modelo': [], 'R2_train': [], 'R2_test': [], 'MSE_train': [], 'MSE_test': []}

for name, model in models.items():
    X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    results['Modelo'].append(name)
    results['R2_train'].append(r2_score(y_train, y_pred_train))
    results['R2_test'].append(r2_score(y_test, y_pred_test))
    results['MSE_train'].append(mean_squared_error(y_train, y_pred_train))
    results['MSE_test'].append(mean_squared_error(y_test, y_pred_test))

results['Modelo'].append("Model 2 (3 num)")
results['R2_train'].append(r2_score(y_train_2, y_pred_train_2))
results['R2_test'].append(r2_score(y_test_2, y_pred_test_2))
results['MSE_train'].append(mean_squared_error(y_train_2, y_pred_train_2))
results['MSE_test'].append(mean_squared_error(y_test_2, y_pred_test_2))

results_df = pd.DataFrame(results)
print(results_df)

fig, axes = plt.subplots(1,2, figsize=(12,5))

# R2 plot
axes[0].bar(results_df['Modelo'], results_df['R2_train'], alpha=0.7, label='Train')
axes[0].bar(results_df['Modelo'], results_df['R2_test'], alpha=0.7, label='Test')
axes[0].set_ylim(-0.5,1)
axes[0].set_title('R² Train vs Test')
axes[0].legend()
axes[0].set_ylabel('R²')
axes[0].tick_params(axis='x', rotation=30)

# MSE plot
axes[1].bar(results_df['Modelo'], results_df['MSE_train'], alpha=0.7, label='Train')
axes[1].bar(results_df['Modelo'], results_df['MSE_test'], alpha=0.7, label='Test')
axes[1].set_title('MSE Train vs Test')
axes[1].legend()
axes[1].set_ylabel('MSE')
axes[1].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()


Podemos ver que Ridge mejoró al modelo 2, con un R2 más alto y un MSE mucho menor. Al contrario, Lasso *overfittea*.

Pero estamos usando lambdas (en este caso alphas) para nada optimizados, por lo que vamos a buscar valores más óptimos para este hiperparámetro. Podríamos usar cross validation, pero vamos a hacer un barrido de forma que veamos el cambio gráficamente:

In [ ]:
alphas = np.logspace(-3, 2, 50)  # valores desde 0.001 hasta 100
r2_train_list = []
r2_test_list = []

r2_train_ridge, r2_test_ridge = [], []
r2_train_lasso, r2_test_lasso = [], []

for a in alphas:
    # Ridge
    ridge = Ridge(alpha=a)
    ridge.fit(X_train, y_train)
    r2_train_ridge.append(r2_score(y_train, ridge.predict(X_train)))
    r2_test_ridge.append(r2_score(y_test, ridge.predict(X_test)))
    
    # Lasso
    lasso = Lasso(alpha=a, max_iter=10000)
    lasso.fit(X_train, y_train)
    r2_train_lasso.append(r2_score(y_train, lasso.predict(X_train)))
    r2_test_lasso.append(r2_score(y_test, lasso.predict(X_test)))


best_alpha_ridge = alphas[np.argmax(r2_test_ridge)]
best_r2_ridge = max(r2_test_ridge)

best_alpha_lasso = alphas[np.argmax(r2_test_lasso)]
best_r2_lasso = max(r2_test_lasso)

print(f"Ridge: mejor alpha = {best_alpha_ridge:.5f}, R² test = {best_r2_ridge:.5f}")
print(f"Lasso: mejor alpha = {best_alpha_lasso:.5f}, R² test = {best_r2_lasso:.5f}")

# --- Graficar ---
plt.figure(figsize=(10,6))
plt.semilogx(alphas, r2_test_ridge, 'b--o', label='R² Test Ridge')
plt.semilogx(alphas, r2_test_lasso, 'r--o', label='R² Test Lasso')

# Marcar los α óptimos
plt.scatter(best_alpha_ridge, best_r2_ridge, color='blue', s=100, marker='x', label=f'Ridge α óptimo')
plt.scatter(best_alpha_lasso, best_r2_lasso, color='red', s=100, marker='x', label=f'Lasso α óptimo')

plt.xlabel('Alpha (log scale)')
plt.ylabel('R²')
plt.title('Barrido de alpha: Ridge vs Lasso')
plt.legend()
plt.grid(True)
plt.show()


Habiendo hecho este análisis, volvemos a graficar:

In [ ]:
models = {
    "Ridge (todas)": Ridge(alpha=best_alpha_ridge),
    "Lasso (todas)": Lasso(alpha=best_alpha_lasso)
}

results = {'Modelo': [], 'R2_train': [], 'R2_test': [], 'MSE_train': [], 'MSE_test': []}

for name, model in models.items():
    X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    results['Modelo'].append(name)
    results['R2_train'].append(r2_score(y_train, y_pred_train))
    results['R2_test'].append(r2_score(y_test, y_pred_test))
    results['MSE_train'].append(mean_squared_error(y_train, y_pred_train))
    results['MSE_test'].append(mean_squared_error(y_test, y_pred_test))

results['Modelo'].append("Model 2 (3 num)")
results['R2_train'].append(r2_score(y_train_2, y_pred_train_2))
results['R2_test'].append(r2_score(y_test_2, y_pred_test_2))
results['MSE_train'].append(mean_squared_error(y_train_2, y_pred_train_2))
results['MSE_test'].append(mean_squared_error(y_test_2, y_pred_test_2))

results_df = pd.DataFrame(results)
print(results_df)

fig, axes = plt.subplots(1,2, figsize=(12,5))

# R2 plot
axes[0].bar(results_df['Modelo'], results_df['R2_train'], alpha=0.7, label='Train')
axes[0].bar(results_df['Modelo'], results_df['R2_test'], alpha=0.7, label='Test')
axes[0].set_ylim(-0.5,1)
axes[0].set_title('R² Train vs Test')
axes[0].legend()
axes[0].set_ylabel('R²')
axes[0].tick_params(axis='x', rotation=30)

# MSE plot
axes[1].bar(results_df['Modelo'], results_df['MSE_train'], alpha=0.7, label='Train')
axes[1].bar(results_df['Modelo'], results_df['MSE_test'], alpha=0.7, label='Test')
axes[1].set_title('MSE Train vs Test')
axes[1].legend()
axes[1].set_ylabel('MSE')
axes[1].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

Ahora se ve mucho mejor! Lasso ya no overfittea, aunque Ridge sigue siendo ligeramente mejor, teniendo mejor R2 y menor MSE.

Ahora, vamos a ver las variables que priorizaron estos modelos:

In [ ]:
top_n = 15

X_num = X[numeric_cols].columns
X_cat = encoder.get_feature_names_out(categorical_cols)
all_features = np.concatenate([X_num, X_cat])

# --- Coeficientes Ridge ---
ridge_coef = pd.Series(models["Ridge (todas)"].coef_, index=all_features)
ridge_coef_sorted = ridge_coef.sort_values(key=abs, ascending=False)
ridge_top = ridge_coef_sorted.head(top_n)

# --- Coeficientes Lasso ---
lasso_coef = pd.Series(models["Lasso (todas)"].coef_, index=all_features)
lasso_coef_sorted = lasso_coef.sort_values(key=abs, ascending=False)
lasso_top = lasso_coef_sorted.head(top_n)

fig, axes = plt.subplots(1,2, figsize=(14,6))

# Ridge
axes[0].barh(ridge_top.index[::-1], ridge_top.values[::-1], color='skyblue')
axes[0].set_title("Top 15 coeficientes Ridge")
axes[0].set_xlabel("Coeficiente")
axes[0].set_ylabel("Variable")

# Lasso
axes[1].barh(lasso_top.index[::-1], lasso_top.values[::-1], color='salmon')
axes[1].set_title("Top 15 coeficientes Lasso")
axes[1].set_xlabel("Coeficiente")
axes[1].set_ylabel("Variable")

plt.tight_layout()
plt.show()

# Conclusión

Ridge fue la mejor opción, aunque logramos mejorar mucho el rendimiento de Lasso modificando lambda. Ridge eligió priorizar ciertos nombres de autos que seguramente son más caros que el resto, lo cual es algo a mejorar (no le veo mucho sentido si lo que queremos es predecir el precio de un nuevo auto). Al contrario, Lasso priorizó especificaciones de los vehículos.

Algo destacable es que ambos encontraron que la localización del motor (específicamente en la parte trasera) y el tamaño de este (lo cual descubrimos con la matriz de correlación) expresado en pulgadas cúbicas son los factores más determinantes a la hora de predecir el precio de un vehículo.